In [3]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import shutil
import os
import logging
import sys
sys.path.append('..')

from textblob import TextBlob
from pprint import pprint
from sklearn.metrics import classification_report

from transformers import AutoModelForSequenceClassification

from finbert.finbert import *
import finbert.utils as tools

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', -1)

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.ERROR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
standard_dir = project_dir.parent.parent

# ! TODO: PUT THE PREPROCESSING CODE HERE AND STORE PROCESSED DF

# Prepare model

In [65]:
lm_path = project_dir/'models'/'language_model'/'finbertTRC2'
final_model_dir = project_dir/'models'/'classification_model'
data_dir = standard_dir/"Data"
# classification_data_path= standard_dir/'Data'/'sentiment_data'

In [21]:
data = pd.read_csv(standard_dir/"Data"/"processed_data.csv")

In [42]:
from sklearn.model_selection import train_test_split

In [43]:
y = data['category']
X = data[[col for col in data.columns if col != "category"]]

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

c:\ProgramData\Anaconda3\envs\finbert\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
c:\ProgramData\Anaconda3\envs\finbert\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your c

In [46]:
train_data = X_train.copy()
train_data["category"] = y_train

In [49]:
train_data.to_csv(standard_dir/"Data"/"train.csv")

In [50]:
test_data = X_test.copy()
test_data["category"] = y_test

In [63]:
test_data.to_csv(standard_dir/"Data"/"eval.csv")
test_data.to_csv(standard_dir/"Data"/"validation.csv")

In [53]:
test_data.shape

(68739, 14)

In [31]:
vals = data.reset_index()[["index", "description", "category"]].values

## Configure training parameters

In [12]:
# Clean the cl_path
try:
    shutil.rmtree(cl_path) 
except:
    pass

## load the pretrained language model trained on Reuters TRC2
bertmodel = AutoModelForSequenceClassification.from_pretrained(
    lm_path, cache_dir=None, num_labels=30)

Some weights of the model checkpoint at c:\Programming\MoneyLion\standard\advanced\finBERT_transaction_classification\models\language_model\finbertTRC2 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
S

In [17]:
from finbert.utils import DataProcessor

In [68]:
config = Config(
    data_dir=data_dir,
    model_dir=final_model_dir,
    bert_model=bertmodel,
    num_train_epochs=5,
    max_seq_length=64,
    train_batch_size=32,
    learning_rate=2e-5,
    output_mode="classification",
    warm_up_proportion=0.2,
    local_rank=-1,
    discriminate=True,
    gradual_unfreeze=True
)

In [69]:
finbert = FinBert(config)

In [77]:
finbert.prepare_model(label_list=y.unique().tolist(), transaction_classification=True)

07/16/2024 17:18:07 - INFO - finbert.finbert -   device: cpu n_gpu: 0, distributed training: False, 16-bits training: False


### Fine-tune the model

In [85]:
# Get the training examples
train_data = finbert.get_data('train', label_colname='category')

In [93]:
model = finbert.create_the_model()

In [94]:
trained_model = finbert.train(train_examples = train_data, model = model)

07/16/2024 17:26:04 - INFO - finbert.utils -   *** Example ***
07/16/2024 17:26:04 - INFO - finbert.utils -   guid: 0
07/16/2024 17:26:04 - INFO - finbert.utils -   tokens: [CLS] online reference # 01 ##0 ##20 ##3 transfer to money market 05 : 03 ##a # 64 ##10 [SEP]
07/16/2024 17:26:04 - INFO - finbert.utils -   input_ids: 101 3784 4431 1001 5890 2692 11387 2509 4651 2000 2769 3006 5709 1024 6021 2050 1001 4185 10790 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/16/2024 17:26:04 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/16/2024 17:26:04 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/16/2024 17:26:04 - INFO - finbert.utils -   label: Transfer Debit (id = 27)
07/16/2024 17:26:23 - INFO

Iteration:   0%|          | 0/5013 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [02:43<?, ?it/s]


KeyboardInterrupt: 

---

In [54]:
base_model = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(base_model, do_lower_case=True)